In [2]:
import pandas as pd
import numpy as np

read_csv(  
    &emsp;file = "path",  
    &emsp;sep = "thing",  
    &emsp;header = 0, if there is no header then header = None  
    &emsp;names = ["columnname", "columnname"] - column names,  
    &emsp;index_col = "column to use as index",  
    &emsp;usecols = [list of columns to keep in df],  
    &emsp;dtype = {dictionary of column names and datatypes},  
    &emsp;parse_dates = [column to parse] - converts date string into datetimes,  
    &emsp;infer_datetime_format = True,  
    &emsp;na_values = [list of strings to recognize as NaN],  
    &emsp;nrows = int, number of rows to read in  
    &emsp;skiprows = [list of ints] line numbers to skip (accepts lambda functions)  
    &emsp;converters = {dictionary with column names and functions to apply}  
)

In [11]:
columns = ["Date", "Store_Number", "Transaction_Count"]
target_pct = lambda x: (x["Transaction_Count"]) / 2500
met_target = lambda x: (x["target_pct"]) >= 1
bonus_payable = lambda x: (x["met_target"]) * 100
month = (lambda x: x["date"].dt.month,)
day_of_week = lambda x: x["date"].dt.day_of_week

df = pd.read_csv(
    "../data/transactions.csv",
    names=columns,
    skiprows=[0, 1],
    parse_dates=["Date"],
    dtype=(
        ({"Store_Number": "Int8", "Transaction_Count": "Int16"})
        .assign(
            target_pct=lambda x: (x["Transaction_Count"]) / 2500,
            met_target=lambda x: ((x["Transaction_Count"]) / 2500) >= 1,
            bonus_payable=lambda x: (((x["Transaction_Count"]) / 2500) >= 1) * 100,
            month=lambda x: x["Date"].dt.month,
            day_of_week=lambda x: x["Date"].dt.dayofweek,
        )
        .astype({"target_pct": "Float32", "month": "Int8", "day_of_week": "Int8"})
    ),
)

df.head()

AttributeError: 'dict' object has no attribute 'assign'

In [23]:
string_slice = lambda x: int(x[-2:])

df = (
    pd.read_csv(
        "../data/transactions.csv",
        names=["Date", "Store_Number", "Transaction_Count"],
        skiprows=[0, 1],
        parse_dates=["Date"],
        dtype={"Store_Number": "Int8", "Transaction_Count": "Int16"},
    )
    .assign(
        target_pct=lambda x: (x["Transaction_Count"]) / 2500,
        met_target=lambda x: ((x["Transaction_Count"]) / 2500) >= 1,
        bonus_payable=lambda x: (((x["Transaction_Count"]) / 2500) >= 1) * 100,
        month=lambda x: x["Date"].dt.month,
        day_of_week=lambda x: x["Date"].dt.dayofweek,
    )
    .astype({"target_pct": "Float32", "month": "Int8", "day_of_week": "Int8"})
)
df.head(3)

,Date,Store_Number,Transaction_Count,target_pct,met_target,bonus_payable,month,day_of_week
0,2013-01-02,1,2111,0.8444,False,0,1,2
1,2013-01-02,2,2358,0.9432,False,0,1,2
2,2013-01-02,3,3487,1.3948,True,100,1,2


The SQLAlchemy library lets you connec to SQL databases

appending stacks the rows from multiple dataframe with the same column structure
joinging adds related columns from one dataframe to another based on common values

append with concat
pd.concat([df1, df2]) df2 on the bottom of df1

ignore_index=True to reset the index on the concat dataframes

left_df.merge(right_df, how, left_on, right_on)

join types inner, left, right, outer
right join is almost never used

inner join returns records that exist in BOTH tables, and exludes unmatches records from either table  
example left table is retail right is transactions. there is a date in the retail table that is not in the transactions table, that row will not make it into the final dataframe (inner join in ven diagram is the overlapping part)

left join returns all records from the left table and any matchingg records from the right table

right join returns all records from the right table and any matching records from the left table (mirror image of the left join)

outer join returns all records from both tables, including non-matching records

In [25]:
stores = pd.read_csv("../data/stores.csv")
retail = pd.read_csv("../data/retail_2016_2017.csv")

In [26]:
stores.head(2)

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13


In [27]:
retail.head(2)

,id,date,store_nbr,family,sales,onpromotion
0,1945944,2016-01-01,1,AUTOMOTIVE,0.0,0
1,1945945,2016-01-01,1,BABY CARE,0.0,0


In [38]:
dfmerge = stores.merge(
    retail,
    how="left",
    left_on=["store_nbr"],
    right_on=["store_nbr"],
)
dfmerge["dt_date"] = pd.to_datetime(dfmerge["date"])
dfmerge.set_index("dt_date")

,store_nbr,city,state,type,cluster,id,date,family,sales,onpromotion
dt_date,,,,,,,,,,
2016-01-01,1,Quito,Pichincha,D,13,1945944,2016-01-01,AUTOMOTIVE,0.000,0
2016-01-01,1,Quito,Pichincha,D,13,1945945,2016-01-01,BABY CARE,0.000,0
2016-01-01,1,Quito,Pichincha,D,13,1945946,2016-01-01,BEAUTY,0.000,0
2016-01-01,1,Quito,Pichincha,D,13,1945947,2016-01-01,BEVERAGES,0.000,0
2016-01-01,1,Quito,Pichincha,D,13,1945948,2016-01-01,BOOKS,0.000,0
...,...,...,...,...,...,...,...,...,...,...
2017-08-15,54,El Carmen,Manabi,C,3,3000751,2017-08-15,POULTRY,59.619,0
2017-08-15,54,El Carmen,Manabi,C,3,3000752,2017-08-15,PREPARED FOODS,94.000,0
2017-08-15,54,El Carmen,Manabi,C,3,3000753,2017-08-15,PRODUCE,915.371,76


In [39]:
dfmerge.index.dtype

dtype('int64')